In [645]:
import datetime
import statistics

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.display import display
from scipy.stats import chi2_contingency
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression


%matplotlib inline
plt.rcParams["figure.figsize"] = (12,6)

In [646]:
df_og = pd.read_csv('data/BADS_WS2021_known.csv')
# df_uk = pd.read_csv('data/BADS_WS2021_unknown.csv')

X_train, X_test, y_train, y_test = train_test_split(df_og['return'], df_og.drop('return', axis=1), test_size=0.001)
y_test.insert(len(y_test.columns), 'return', X_test)
df = y_test

In [647]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 76193 to 3677
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   order_item_id  100 non-null    int64  
 1   order_date     100 non-null    object 
 2   delivery_date  90 non-null     object 
 3   item_id        100 non-null    int64  
 4   item_size      100 non-null    object 
 5   item_color     100 non-null    object 
 6   brand_id       100 non-null    int64  
 7   item_price     100 non-null    float64
 8   user_id        100 non-null    int64  
 9   user_title     100 non-null    object 
 10  user_dob       94 non-null     object 
 11  user_state     100 non-null    object 
 12  user_reg_date  100 non-null    object 
 13  return         100 non-null    int64  
dtypes: float64(1), int64(5), object(8)
memory usage: 11.7+ KB
None


In [648]:
df.item_size.value_counts()
df.item_color.value_counts()
df.user_title.value_counts()
df.user_state.value_counts()

North Rhine-Westphalia    31
Lower Saxony              14
Bavaria                   10
Hesse                      9
Rhineland-Palatinate       6
Schleswig-Holstein         6
Thuringia                  5
Brandenburg                4
Hamburg                    3
Saxony                     3
Baden-Wuerttemberg         3
Bremen                     2
Berlin                     2
Saxony-Anhalt              1
Saarland                   1
Name: user_state, dtype: int64

In [649]:
df.user_title = df.user_title.astype('category')
df.item_size = df.item_size.astype('category')
df.item_color = df.item_color.astype('category')
df.user_state = df.user_state.astype('category')


df['order_date'] = pd.to_datetime(df['order_date'])
df['delivery_date'] = pd.to_datetime(df['delivery_date'])
df['user_reg_date'] = pd.to_datetime(df['user_reg_date'])
df['user_dob'] = pd.to_datetime(df['user_dob'])

df['return'] = df['return'].astype('bool')
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 76193 to 3677
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   order_item_id  100 non-null    int64         
 1   order_date     100 non-null    datetime64[ns]
 2   delivery_date  90 non-null     datetime64[ns]
 3   item_id        100 non-null    int64         
 4   item_size      100 non-null    category      
 5   item_color     100 non-null    category      
 6   brand_id       100 non-null    int64         
 7   item_price     100 non-null    float64       
 8   user_id        100 non-null    int64         
 9   user_title     100 non-null    category      
 10  user_dob       94 non-null     datetime64[ns]
 11  user_state     100 non-null    category      
 12  user_reg_date  100 non-null    datetime64[ns]
 13  return         100 non-null    bool          
dtypes: bool(1), category(4), datetime64[ns](4), float64(1), int64(4)
memo

In [650]:
# finding missing values

print(df.isnull().sum())
# sns.heatmap(df.isnull(), cbar=False)  # quick visualization of the missing values in our data set

order_item_id     0
order_date        0
delivery_date    10
item_id           0
item_size         0
item_color        0
brand_id          0
item_price        0
user_id           0
user_title        0
user_dob          6
user_state        0
user_reg_date     0
return            0
dtype: int64


In [651]:
# check relation between null values in delivery_date and return value
# assumption: delivery_date=null --> item not delivered --> item was returned before it was delivered

df.loc[df.delivery_date.isnull(), 'return'].value_counts()
# all orders where delivery_date is null (missing), were not returned, assumption was wrong.
# I assume that all those orders were delivered


False    10
Name: return, dtype: int64

In [652]:
print('amount of orders delivered on 1994-12-31: {}'.format(df_og.loc[df_og.delivery_date=='1994-12-31', 'delivery_date'].count()))
print('[delivery_date, amount] of orders where the delivery_date is before the order_date: {}'.format(df_og.loc[df_og.delivery_date<df_og.order_date, 'delivery_date'].value_counts()))

# all delivery_date where the delivery date is 1994-12-31 need to be replaced changed together with the missing values

amount of orders delivered on 1994-12-31: 1072
[delivery_date, amount] of orders where the delivery_date is before the order_date: 1994-12-31    1072
Name: delivery_date, dtype: int64


In [653]:
df.head()

,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return
76193,76194,2016-08-21,2016-08-22,1697,39,brown,50,99.95,43163,Mrs,1962-05-31,Bremen,2015-02-17,True
3590,3591,2016-06-26,2016-06-27,214,xxl,denim,49,44.90,19989,Mrs,1974-12-15,Bavaria,2015-08-12,True
90108,90109,2016-09-02,2016-09-05,2131,41,brown,44,175.00,46444,Mr,1941-03-25,Schleswig-Holstein,2016-09-03,True
91071,91072,2016-09-03,2016-09-04,1595,38,anthracite,17,199.90,46595,Mrs,NaT,Saxony,2016-03-25,False
77266,77267,2016-08-22,2016-08-24,1888,43,black,1,199.90,44136,Mrs,1969-12-29,Saarland,2015-02-17,True


In [654]:
# change dates
# calculate delivery_times

In [655]:
# df.insert(3, 'delivery_time', [0]*len(df))

In [656]:
# replace delivery_times with None where delivery_date=='1994-12-31' (negative delivery_time) in order to replace the value with mean
df.loc[df.delivery_date=='1994-12-31', 'delivery_date'] = None

#create new column delivery_time, for null values in delivery_date, the mean delivery_time will be placed
df['delivery_time'] = [(d_date - o_date).days for d_date, o_date in zip(df.delivery_date, df.order_date)]

delivery_time_mean = round(df.delivery_time.mean(skipna=True))
df.delivery_time.fillna(delivery_time_mean, inplace=True)


In [657]:
# check if there is a relation between missing dob and user_title (i.e. companies dont have birth dates)
df['user_title'][df.user_dob.isnull()].value_counts()

# no pattern recognizable

Mrs       5
Mr        1
Family    0
Name: user_title, dtype: int64

In [658]:
df['user_age'] = [(o_date - dob).days / 365 for o_date, dob in zip(df.order_date, df.user_dob)]
user_age_mean = round(df.user_age.mean(skipna=True))
df.user_age.fillna(user_age_mean)
df.user_age = df.user_age.round()
df.head()


,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return,delivery_time,user_age
76193,76194,2016-08-21,2016-08-22,1697,39,brown,50,99.95,43163,Mrs,1962-05-31,Bremen,2015-02-17,True,1.0,54.0
3590,3591,2016-06-26,2016-06-27,214,xxl,denim,49,44.90,19989,Mrs,1974-12-15,Bavaria,2015-08-12,True,1.0,42.0
90108,90109,2016-09-02,2016-09-05,2131,41,brown,44,175.00,46444,Mr,1941-03-25,Schleswig-Holstein,2016-09-03,True,3.0,75.0
91071,91072,2016-09-03,2016-09-04,1595,38,anthracite,17,199.90,46595,Mrs,NaT,Saxony,2016-03-25,False,1.0,NaN
77266,77267,2016-08-22,2016-08-24,1888,43,black,1,199.90,44136,Mrs,1969-12-29,Saarland,2015-02-17,True,2.0,47.0


In [659]:
# check if there are negative price_values
len(df.loc[df.item_price < 0, 'item_price'])

0

In [660]:
# y = df['return']
# X = df.drop(['order_item_id', 'order_date', 'delivery_date', 'user_id', 'user_dob','return'], axis=1)
#
# X_train, X_test, y_train, y_test = train_test_split(X, y)

In [661]:

# estimator = LogisticRegression()
# parameter = {"penalty":["none", "l2"]}
# logit = GridSearchCV(estimator, parameter)
# logit.fit(X_train, y_train)

In [662]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 76193 to 3677
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   order_item_id  100 non-null    int64         
 1   order_date     100 non-null    datetime64[ns]
 2   delivery_date  89 non-null     datetime64[ns]
 3   item_id        100 non-null    int64         
 4   item_size      100 non-null    category      
 5   item_color     100 non-null    category      
 6   brand_id       100 non-null    int64         
 7   item_price     100 non-null    float64       
 8   user_id        100 non-null    int64         
 9   user_title     100 non-null    category      
 10  user_dob       94 non-null     datetime64[ns]
 11  user_state     100 non-null    category      
 12  user_reg_date  100 non-null    datetime64[ns]
 13  return         100 non-null    bool          
 14  delivery_time  100 non-null    float64       
 15  user_age       94 

In [663]:
res = pd.crosstab(df_og.item_size, df_og['return'])
res['odds_ratio'] = res[0] / res[1]
res.sort_values('odds_ratio', inplace=True)
display(res.to_stata)

<bound method DataFrame.to_stata of return      0  1  odds_ratio
item_size                   
4034        0  1         0.0
3132        0  1         0.0
2+          0  2         0.0
80          0  2         0.0
84          0  1         0.0
...        .. ..         ...
49          1  0         inf
3432       13  0         inf
105         1  0         inf
3634        1  0         inf
3834        1  0         inf

[102 rows x 3 columns]>

In [664]:
# remap_dict = {
#     '4034': '4034, 3132, 2+, 80, 84, 12+',
#     '3132': '4034, 3132, 2+, 80, 84, 12+',
#     '2+': '4034, 3132, 2+, 80, 84, 12+',
#     '80': '4034, 3132, 2+, 80, 84, 12+',
#     '84': '4034, 3132, 2+, 80, 84, 12+',
#     '12+': '4034, 3132, 2+, 80, 84, 12+',
#     '22'
# }

In [665]:
res = pd.crosstab(df_og.user_state, df_og['return'])
res['odds_ratio'] = res[0] / res[1]
res.sort_values('odds_ratio', inplace=True)
res

return,0,1,odds_ratio
user_state,,,
Mecklenburg-Western Pomerania,855,785,1.089172
Saxony,1647,1500,1.098000
Schleswig-Holstein,2929,2602,1.125673
Bavaria,7280,6392,1.138924
Lower Saxony,8110,7039,1.152152
Berlin,2201,1892,1.163319
Brandenburg,1201,1030,1.166019
Saxony-Anhalt,606,516,1.174419
Saarland,484,412,1.174757


In [666]:
remap_dict = {
    'Mecklenburg-Western Pomerania' : 'user_state_group_5',
    'Saxony' : 'user_state_group_5',
    'Schleswig-Holstein': 'user_state_group_5',
    'Bavaria': 'user_state_group_5',
    'Lower Saxony': 'user_state_group_1',
    'Berlin': 'user_state_group_1',
    'Brandenburg': 'user_state_group_1',
    'Saxony-Anhalt': 'user_state_group_2',
    'Saarland': 'user_state_group_2',
    'Hesse': 'user_state_group_2',
    'Rhineland-Palatinate': 'user_state_group_2',
    'Baden-Wuerttemberg': 'user_state_group_2',
    'North Rhine-Westphalia': 'user_state_group_3',
    'Thuringia': 'user_state_group_3',
    'Hamburg': 'user_state_group_3',
    'Bremen': 'user_state_group_4'
}

df_og['user_state_merged'] = df_og['user_state'].map(remap_dict).astype('category')

In [667]:
ct_orig = pd.crosstab(df_og['return'], df_og['user_state'])
stat_orig, _, _, _ = chi2_contingency(ct_orig)

# Testing the modified EMPS_A encoding
ct_merged = pd.crosstab(df_og['return'], df_og['user_state_merged'])
stat_merged, _, _, _ = chi2_contingency(ct_merged)

# Comparing test results
print('Chi-square statistic (original): {:.4f}'.format(stat_orig))
print('Chi-square statistic (encoded):  {:.4f}'.format(stat_merged))


Chi-square statistic (original): 45.5548
Chi-square statistic (encoded):  42.3938


In [668]:
res = pd.crosstab(df_og.item_color, df_og['return'])
res['odds_ratio'] = res[0] / res[1]
res.sort_values('odds_ratio', inplace=True)
res['bin'] = pd.cut(res.odds_ratio,
                       bins=[0,0.7, 1.49, 2.1, 3.5, 5, 11, float('inf')],
                       labels=[1,2,3,4,5,6,7])

df_og.insert(6, 'item_color_bin', [0]*len(df_og))
for idx, row in res.iterrows():
    df_og.loc[df_og.item_color==idx, 'item_color_bin'] = row.bin


ct_orig = pd.crosstab(df_og['return'], df_og['item_color'])
stat_orig, _, _, _ = chi2_contingency(ct_orig)

# Testing the modified EMPS_A encoding
ct_merged = pd.crosstab(df_og['return'], df_og['item_color_bin'])
stat_merged, _, _, _ = chi2_contingency(ct_merged)

# Comparing test results
print('Chi-square statistic (original): {:.4f}'.format(stat_orig))
print('Chi-square statistic (encoded):  {:.4f}'.format(stat_merged))

Chi-square statistic (original): 777.3392
Chi-square statistic (encoded):  546.3846


In [669]:
df_og.head()

,order_item_id,order_date,delivery_date,item_id,item_size,item_color,item_color_bin,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return,user_state_merged
0,1,2016-06-22,2016-06-27,643,38,navy,4.0,30,49.90,30822,Mrs,1969-04-17,Saxony,2016-06-23,0,user_state_group_5
1,2,2016-06-22,NaN,337,152,grey,2.0,30,19.95,30822,Mrs,1969-04-17,Saxony,2016-06-23,0,user_state_group_5
2,3,2016-06-22,2016-06-27,270,xxl,grey,2.0,49,79.90,30823,Mrs,1970-04-22,Baden-Wuerttemberg,2015-03-15,1,user_state_group_2
3,4,2016-06-22,2016-06-27,142,xxl,grey,2.0,49,99.90,30823,Mrs,1970-04-22,Baden-Wuerttemberg,2015-03-15,0,user_state_group_2
4,5,2016-06-22,2016-06-27,561,xxl,grey,2.0,3,14.90,30823,Mrs,1970-04-22,Baden-Wuerttemberg,2015-03-15,1,user_state_group_2
